# Analyze dDEMs as a function of land cover type, canopy height, and terrain parameters

In [6]:
import os
import matplotlib.pyplot as plt
import xdem 
import geoutils as gu

In [ ]:
# pipeline outputs
job_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/MCS/20240420/skysat_snow'
ddem_fn = os.path.join(job_dir, 'final_align', 'final_dDEM.tif')
ref_dem_fn = os.path.join(job_dir, 'refdem_EPSG32611.tif')
roads_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'roads_mask.tif')
snow_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'snow_mask.tif')
ss_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'stable_surfaces_mask.tif')
trees_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'trees_mask.tif')

# output folder
out_dir = os.path.join(job_dir, 'ddem_analysis')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

## Load dDEM and reference DEM

In [8]:
ddem = xdem.DEM(ddem_fn)
refdem = xdem.DEM(ref_dem_fn)
refdem = refdem.reproject(ddem)

## dDEM vs. land cover type

In [ ]:
fig_fn = os.path.join(out_dir, 'ddem_vs_land_cover_types.png')
plt.rcParams.update({'font.sans-serif': 'Arial', 'font.size': 12})

if not os.path.exists(fig_fn):
    # Load land cover masks
    roads_mask = gu.Raster(roads_mask_fn, load_data=True)
    roads_mask = roads_mask.reproject(ddem)
    snow_mask = gu.Raster(snow_mask_fn, load_data=True)
    snow_mask = snow_mask.reproject(ddem)
    ss_mask = gu.Raster(ss_mask_fn, load_data=True)
    ss_mask = ss_mask.reproject(ddem)
    trees_mask = gu.Raster(trees_mask_fn, load_data=True)
    trees_mask = trees_mask.reproject(ddem)

    # Create dataframe
    df = pd.DataFrame({'dDEM': ddem.data.ravel(),
                       'roads_mask': roads_mask.data.ravel(),
                       'snow_mask': snow_mask.data.ravel(),
                       'ss_mask': ss_mask.data.ravel(),
                       'trees_mask': trees_mask.data.ravel()})
    df.dropna(inplace=True)

    # Plot histogram
    bins = np.linspace(-10, 10, 200)
    cols = ['snow_mask', 'ss_mask', 'trees_mask', 'roads_mask'] 
    colors = [(55/255, 126/255, 184/255, 1), # snow
              (153/255, 153/255, 153/255, 1), # stable surfaces
              (77/255, 175/255, 74/255, 1), # trees
              (166/255, 86/255, 40/255, 1)] # roads
    fig, ax = plt.subplots(2, 2, figsize=(10,5))
    ax = ax.flatten()
    i=0
    for col, color in zip(cols, colors):
        df_sub = df.loc[df[col]==1, 'dDEM']
        ax[i].hist(df_sub.values, bins=bins, color=color, alpha=0.8, label=col)
        ax[i].set_xlim(-5, 5)
        ax[i].set_title(col.replace('ss_mask', 'stable surfaces').replace('_mask', ''))
        i+=1

    fig.tight_layout()
    plt.show()

    # Save figure
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)
    
else:
    print('Figure already exists, skipping.')

## dDEM vs. terrain parameters

In [ ]:
fig_fn = os.path.join(out_dir, 'ddem_vs_terrain_parameters.png')

if not os.path.exists(fig_fn):
    
    # Calculate terrain parameters from reference DEM
    slope = refdem.slope()
    aspect = refdem.aspect()
    # Compile all stats into dataframe
    terrain_cols = ['elevation', 'slope', 'aspect']
    stats_df = pd.DataFrame(columns=terrain_cols + ['dDEM'])
    for raster, col in zip([refdem, slope, aspect], terrain_cols):
        stats_df[col] = raster.data.ravel()
        # Create bins for column
        stats_df[col + '_bin'] = pd.cut(stats_df[col], bins=25, precision=0)
    stats_df['dDEM'] = ddem.data.ravel() 
    stats_df.dropna(inplace=True)
    stats_df.reset_index(drop=True, inplace=True)
    
    # Plot
    fig, ax = plt.subplots(1,3, figsize=(16,6))
    for i, col in enumerate(terrain_cols):
        sns.boxplot(stats_df, x=col + '_bin', y='dDEM', showfliers=False, ax=ax[i])
        ax[i].set_title(col)
        if i==0:
            ax[i].set_ylabel('dDEM [m]')
        ax[i].set_xlabel(col)
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=90)
        ax[i].axhline(0, color='k')
    fig.tight_layout()
    plt.show()
    # Save figure
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    print('Figure saved to file:', fig_fn)
    
else:
    print('Figure already exists, skipping.')

In [ ]:
# Plot SNOTEL
snotel_fn = os.path.join(job_dir, '..', '..', 'snotel', 'MCS_2020-01-01_2024-06-07_adj.csv')
snotel = pd.read_csv(snotel_fn)
snotel['datetime'] = pd.to_datetime(snotel['datetime'])

fig = plt.figure(figsize=(8,6))
plt.plot(snotel['datetime'], snotel['SWE_m'], '-', color='purple', label='SWE')
plt.plot(snotel['datetime'], snotel['SNWD_m'], '-', color='blue', label='Snow depth')
plt.axvline(np.datetime64('2024-04-20'), color='k', linestyle='--', markersize=15, label='SkySat date')
skysat_sd = snotel.loc[snotel['datetime'].dt.date == np.datetime64('2024-04-20'), 'SNWD_m'].values[0]
plt.text(np.datetime64('2024-04-15'), 2.6, f'Snow depth = {np.round(skysat_sd, 3)} m', ha='right')
plt.xlim(np.datetime64('2023-11-01'), np.datetime64('2024-07-01'))
plt.ylabel('meters')
plt.grid()
plt.legend(loc='best')
plt.show()

# fig_fn = os.path.join(data_dir, '..', 'snotel', 'SNWD_SWE_timeseries.png')
# fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
# print('Figure saved to file:', fig_fn)

In [ ]:
# Plot SNOTEL location on orthoimage
from shapely.wkt import loads

ortho_fn = os.path.join(job_dir, '..', 'MCS_20240420_4band_mosaic.tif')
ortho = xr.open_dataset(ortho_fn)
crs = ortho.rio.crs
ortho = xr.where(ortho==0, np.nan, ortho / 1e4)
ortho = ortho.rio.write_crs(crs)
snotel_info_fn = os.path.join(job_dir, '..', '..', 'snotel', 'MCS_SNOTEL_site_info.csv')
snotel_info = pd.read_csv(snotel_info_fn)
snotel_info['geometry'] = snotel_info['geometry'].apply(loads)
snotel_info = gpd.GeoDataFrame(snotel_info, geometry='geometry', crs='EPSG:4326')
snotel_info = snotel_info.to_crs(ortho.rio.crs)

fig = plt.figure(figsize=(6,6))
plt.imshow(np.dstack([ortho.isel(band=2).band_data.data, 
                      ortho.isel(band=1).band_data.data, 
                      ortho.isel(band=0).band_data.data]) * 0.5,
           extent=(np.min(ortho.x)/1e3, np.max(ortho.x)/1e3, 
                   np.min(ortho.y)/1e3, np.max(ortho.y)/1e3))
plt.plot(snotel_info.geometry[0].coords.xy[0][0]/1e3, 
         snotel_info.geometry[0].coords.xy[1][0]/1e3, '*', 
         markerfacecolor='yellow', markeredgecolor='k', markersize=30)
plt.xlabel('Easting [km]')
plt.ylabel('Northing [km]')
plt.show()

fig_fn = snotel_info_fn.replace('.csv', '.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)